In [1]:
import findspark
findspark.init()
from pyspark.mllib.recommendation import ALS, Rating
from pyspark import SparkContext
sc = SparkContext("local", "test")

In [2]:
# Load and parse the data
rating_data = sc.textFile("./data/ml-10M100K/ratings-train.dat")
ratings_train = (rating_data.map(lambda l: l.split(','))
    .map(lambda l: Rating(int(l[0]), int(l[1]), float(l[2])))
           )

In [3]:
validation_data = sc.textFile("./data/ml-10M100K/ratings-validation.dat")
ratings_validation = (rating_data.map(lambda l: l.split(','))
    .map(lambda l: Rating(int(l[0]), int(l[1]), float(l[2])))
           )

#### test.first()

In [4]:
ratings_train.take(3)

[Rating(user=36955, product=21, rating=3.0),
 Rating(user=36955, product=47, rating=5.0),
 Rating(user=36955, product=1079, rating=3.0)]

In [5]:
# Build the recommendation model using Alternating Least Squares
#rank is number of latent variable we want to set
rank = 10
numIterations = 10
lambda_value = 0.01
model = ALS.train(ratings_train, rank, numIterations, lambda_value)

In [6]:
# Evaluate the model on training data
testdata = ratings_validation.map(lambda p: (p[0], p[1]))
predictions = model.predictAll(testdata).map(lambda r: ((r[0], r[1]), r[2]))
ratesAndPreds = ratings_train.map(lambda r: ((r[0], r[1]), r[2])).join(predictions)
MSE = ratesAndPreds.map(lambda r: (r[1][0] - r[1][1])**2).mean()
print("Mean Squared Error = " + str(MSE))

Mean Squared Error = 0.560920921711
